In [7]:
import sys 

sys.path.append("/Users/ngokykhanhthu/Documents/Mental_Health_n_LLMs/")

In [9]:
import re 
# import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 
import os
import json
import subprocess
from datetime import datetime
from mentalLLM.config import RAW_DIR, INTERIM_DIR, PROCESSED_DIR, FINAL_DIR, DATASET, DATA_KEYWORD, CHAT_GPT_QUESTION


os forces python to interract with the operating system 
sys interracts with python interpreter itself

In [29]:
def is_unix_time(object): 
    '''
    Checks to see if a given string is unix time or not 
    
    Args: 
        object that needs to be checked 
    
    Returns: 
        Boolean value, true when it is unix time stamp, false if else 
    '''
    unix_time = re.match(r'^\d{10}\.\d+$', str(object))
    
    if unix_time: 
        return True 
    return False 


In [30]:
is_unix_time(1548319538)

False

cus some strings of random numbs have timestamps as well but they dont have info abt nanoseconds like messages do 

In [31]:
# test that  
is_unix_time(1760895234.572819)

True

In [ ]:
# TODO current results are converted to EST time zone, should change to reflect the users timezone instead
# output is currently not timezone aware 
# shifts for daylight saving time as well 

def convert_unix_time(unix_stamp): 
    """
    Given unix timestamp, convert it to date time object 

    Args:
        unix_stamp (str): time stamp 

    Returns:
        datetime: datetime object in YYYY/MM/DD HH:MM:SS
    """
    return datetime.fromtimestamp(unix_stamp)

In [ ]:
# test that  
convert_unix_time(1465073678)

datetime.datetime(2016, 6, 4, 16, 54, 38)

In [28]:
convert_unix_time(1760895234.572819)

datetime.datetime(2025, 10, 19, 13, 33, 54, 572819)

In [ ]:
# probs drop 
# def check_if_chat_message(unix_stamp, data): 
#     """
#     helper method for create_csv_files
#     given unix stamp and the json list, figure out if it is gibberish or chat message
   
#     Args:
#         unix_stamp (float): an unix timestamp 
#         data (list): the json file 

#     Returns: 
#         True if it is a valid chat message
#         False otherwise  
#     """
#     # TODO 
#     return True 


In [ ]:
def extract_msg(unix_stamp, data): 
    """
    helper method for create_csv_files
    given unix stamp and the json list, extract chat message
    Args:
        unix_stamp (float): an unix timestamp 
        data (list): the json file 

    Returns: 
        str: message
    """
    # curr_index = data.index(unix_stamp) + 1
    strings = []
    
    entered_string = False 
    while curr_index < len(data):
        curr_element = data[curr_index]
        if isinstance(curr_element, str):
            strings.append(curr_element)  
            curr_index += 1
        elif not strings: 
            curr_index += 1 
        else: 
            curr_index = len(data)

    return repr(str(strings[-1]))
       

In [ ]:
# test that 

In [ ]:
# probs drop 
# def classify_sender(message): 
#     """
#     helper method for create_csv_files
#     given unix stamp and the json list, figure out who sent message 
#      Args:
#         message (str): a message extracted 

#     Returns: 
#         str: chatgpt/user 
#     """
#     msg_raw = repr(message)
#     if msg_raw[0] == "\\'": 
#         return "chatgpt" 
#     return "user"
#     # return bool(re.match(r'.*("\""|"\'").*', message[0]))
#     # message_raw = repr(message)[1:]
#     # state =  bool(re.fullmatch(r'.*[\']".*', message_raw))
#     # if state: 
#     #     return "chatgpt"
#     # return "user" 


In [ ]:
def create_csv_files(file_path): 
    """
    given json list file path, extracts chat info and outputs csv file
    
    Args:
        file_path (os.path): path to json file   
    """

    # load json file 
    with open(file_path, "r", encoding='utf-8', errors='strict') as f:
        # type(data) = dict
        data = json.load(f)

    # initialise df 
    rows = []
    
    # meta data stuff 
    title = ""
    create_time = 0 
    update_time = 0

    # go through the json list 
    for i in range(0,len(data)-1, 1): 
        sender = ""
        # extract title 
        if isinstance(data[i], str) and "title" in data[i]: 
            title = data[i+1]
            continue

        if not is_unix_time(data[i]): 
            continue 

        timestamp = data[i]

        # extract file create time 
        if create_time == 0 : 
            create_time = convert_unix_time(timestamp)
            continue 
            
        # extract file latest update time 
        elif update_time == 0:     
            update_time = convert_unix_time(timestamp)
            continue 
    
        # extract timestamp + message 
        
        # if there is a timestamp right afterwards, probs a chatgpt response 
        # elif i + 1 < len(data) and is_unix_time(data[i+1]): 
        #     print("data at ", i, "is ", data[i], "anddd second cond state is ", is_unix_time(data[i+1]))

        #     timestamp =  data[i+1]
        #     sender = "chatgpt"
            
        # # deal w the extra timestamp
        # elif  i - 1  > 0 and is_unix_time(data[i-1]) : 
        #     continue

        # # if there is not, check if gibberish. 
        # elif not check_if_chat_message(timestamp, data): 
        #     continue

        # if not, get the message 
        message = extract_msg(timestamp, data[i+1:]) 

        # if 'sender' != "": 
        #     sender = classify_sender(message)

        # update rows 
        rows.append([title, 
                        create_time, 
                        update_time,
                        "pending", 
                        convert_unix_time(timestamp), 
                        message])
    
    df = pd.DataFrame(rows, columns=['title', 'create_time', "update_time", 'sender', 'timestamp', 'message'])
    df.to_csv(f"{FINAL_DIR}/{file_path[0]}_extracted_chat.csv", index=False)